In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import spacy
import sqlite3, os
import pandas as pd
# import protobuf
from pathlib import Path

In [12]:
%%capture
!pip install protobuf

In [ ]:
os.makedirs("../data/models", exist_ok=True)
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
local_dir = "../data/models/dslim-bert-base-ner"

In [ ]:
# the model was taken from HuggingFace - "https://huggingface.co/dslim/bert-base-NER"

# model_name = "dslim/bert-base-NER"

# tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=local_dir)
# model = AutoModelForTokenClassification.from_pretrained(model_name, cache_dir=local_dir)

# print("Success!!")

c:\Users\aktkr\financial-news-intelligence\.multi_agent_langgraph\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aktkr\financial-news-intelligence\data\models\dslim-bert-base-ner\models--dslim--bert-base-NER. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at dslim/

Success!!


In [21]:
base = Path("../data/models/dslim-bert-base-ner")
snapshot = list(base.glob("**/snapshots/*"))[0]  # first snapshot folder

print("Using snapshot path:", snapshot, "\n---")


tokenizer = AutoTokenizer.from_pretrained(snapshot)
model = AutoModelForTokenClassification.from_pretrained(snapshot)

fin_ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

print("---")
print(tokenizer.tokenize("HDFC Bank RBI repo rate Q2"))
print("---")
print(model.config.id2label)
# print(model.config.label2id)
print("---")

fin_ner("RBI increased the repo rate and HDFC Bank reported strong Q2 results.")

Using snapshot path: ..\data\models\dslim-bert-base-ner\models--dslim--bert-base-NER\snapshots\d1a3e8f13f8c3566299d95fcfc9a8d2382a9affc 
---


Some weights of the model checkpoint at ..\data\models\dslim-bert-base-ner\models--dslim--bert-base-NER\snapshots\d1a3e8f13f8c3566299d95fcfc9a8d2382a9affc were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


---
['HD', '##FC', 'Bank', 'RBI', 're', '##po', 'rate', 'Q', '##2']
---
{0: 'O', 1: 'B-MISC', 2: 'I-MISC', 3: 'B-PER', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-LOC', 8: 'I-LOC'}
---


[{'entity_group': 'ORG',
  'score': np.float32(0.99932396),
  'word': 'RBI',
  'start': 0,
  'end': 3},
 {'entity_group': 'ORG',
  'score': np.float32(0.99931717),
  'word': 'HDFC Bank',
  'start': 32,
  'end': 41}]

In [15]:
conn = sqlite3.connect("../data/financial_news.db")
cur = conn.cursor()
cur.execute(
    """
    SELECT * FROM unique_news ORDER BY id 
    """
)
stories = cur.fetchall()
cur.close()
conn.close()

print("SUCCESS!!")
df = pd.DataFrame(stories, columns=[
    "id", "article_ids", "article_title", "combined_text", "num_articles"
])

df.head()

SUCCESS!!


,id,article_ids,article_title,combined_text,num_articles
0,1,[np.int64(1)],Worried About Inflation? These 3 ETFs Offer Re...,Inflation has slowed but remains a major conce...,1
1,2,[np.int64(2)],Intel’s Black Friday Breakout: Apple Rumors Fu...,A holiday stock surge fueled by credible Apple...,1
2,3,[np.int64(3)],Klarna's Crypto Play: A Plan to Fix Its Profit...,Klarna's launch of a stablecoin is a strategic...,1
3,4,[np.int64(4)],Meta Platforms May Ditch NVIDIA Chips—Here’s W...,Meta Platforms may be looking to alter where i...,1
4,5,[np.int64(5)],SoFi Technologies: From Fintech Speculation to...,SoFi Technologies is proving its long-term val...,1


In [20]:
for i in range(20):
    sample_text = df.loc[i, "combined_text"]
    print(sample_text)
    print(fin_ner(sample_text))
    print("\n---\n")


Inflation has slowed but remains a major concern for many investors; these ETFs can help provide a buffer through the use of TIPS, commodities, or T-Bills.
[]

---

A holiday stock surge fueled by credible Apple partnership rumors signals that the market is finally repricing Intel's manufacturing value.
[{'entity_group': 'ORG', 'score': np.float32(0.99784017), 'word': 'Apple', 'start': 41, 'end': 46}, {'entity_group': 'ORG', 'score': np.float32(0.99887544), 'word': 'Intel', 'start': 111, 'end': 116}]

---

Klarna's launch of a stablecoin is a strategic pivot to lower transaction costs and challenge legacy payment networks, signaling a new era for fintech.
[{'entity_group': 'ORG', 'score': np.float32(0.98230094), 'word': 'Klarna', 'start': 0, 'end': 6}]

---

Meta Platforms may be looking to alter where it spends its AI dollars. See what benefits the firm could reap from Google and Broadcom's chips.
[{'entity_group': 'ORG', 'score': np.float32(0.99667567), 'word': 'Meta Platforms', 'sta